# Segmenting and Clustering Neighborhoods in Toronto

This is the week 3, Peer-graded Assignment for Applied Data Science Capstone in
Coursera.

## One notebook was used for all three parts of the assignment and a markdown clearly label each part. 

# Part 1

## Table of Contents


1.  Scraping the list of postal codes in Canada from the Wikipedia -https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,<br>

2.  Creating a dataframe <br>

3.  Data cleaning: Remove empty cells with "Not assigned" <br>

4.  Using the .shape method to print the number of rows of the dataframe. <br>

#### Step 1 - Scraping the data from Wikipedia

In [1]:
#importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

Using requests library to download the webpage from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, and Saving it in the variable named 'data'.

In [2]:
#Scraping the List of postal codes of Canada
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data=requests.get(url).text

In [3]:
#constructs a BeautifulSoup object from data and stores this as soup
soup = BeautifulSoup(data, 'xml')

In [4]:
#finf the table to store in the variable table
table=soup.find('table')

#### Step 2 - Creating a dataframe

In [5]:
# naming the three columns: PostalCode, Borough, and Neighbourhood 
column_names = ['PostalCode','Borough','Neighbourhood']
#We extract just these three columns: PostalCode, Borough, and Neighborhood from the dataframe
column_names = ['PostalCode','Borough','Neighbourhood']
#creating a DataFrame with the columns 
df = pd.DataFrame(columns = column_names)

In [6]:
# for loop to iterate over tr_cell to get  all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Step 3:  Data cleaning: Remove empty cells with "Not assigned"

In [7]:
df=df[df['Borough']!='Not assigned']

In [8]:
df[df['Neighbourhood']!='Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Step 4: Using the .shape method to print the number of rows of the dataframe

In [9]:
df.shape

(103, 3)

In [10]:
df.dtypes

PostalCode       object
Borough          object
Neighbourhood    object
dtype: object

# End of Part 1

# Part 2

## Table of Contents
1. Get the CSV file  that has the geographical coordinates of each postal code<br>
2. Merge the data scraped in part 1 to the geographical coordinates data using the postal code as a point of merge

#### Step 1- Getting the data with geographical coordinates into a dataframe

In [18]:
# Getting the geographical coordinates of Toronto boroughs from a csv file 
geo_data = pd.read_csv('https://cocl.us/Geospatial_data')
#geo_data.columns=['Postalcode','Latitude','Longitude']
geo_data.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [19]:
#getting the data type
geo_data.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [20]:
#getting the shape
geo_data.shape

(103, 3)

creating a function

In [21]:
def get_coordinate(postal_code):
    # initialising  variable to None
    coords = None
    while(coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        coords = g.latlng
    latitude = coords[0]
    longitude = coords[1]
    return latitude,longitude

#### Step 2- Merge the dataset from part 1 to the geographical coordinates dataset

In [22]:
geo_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
toro_geo_data  = pd.merge(geo_data,df, on="PostalCode")
toro_geo_data.head()

,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [23]:
toro_data=toro_geo_data[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
toro_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
toro_data.shape

(103, 5)

# End of Part 2

# Part 3 Exploring, Segmenting and Clustering the neighborhoods in Toronto.


## Table of Contents
1. Import the necessary Libraries needed for parts 3.<br>
2. Define Foursquare Credentials and Version. <br>
3. Working with only one boroughs that contain the word Toronto.<br>
4. Clustering the Neighborhoods.<br>
5. Examining the Segmentation.<br>

#### Step 1- Import the necessary Libraries needed for parts 3.

In [25]:
# uncomment this line if you have the module named 'folium' and 'geopy'
#!conda install -c conda-forge folium=0.5.0 --yes 
#!pip install geopy

In [26]:
#importing libraries
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
# map rendering library
import folium 
import numpy as np
from pandas.io.json import json_normalize 
#use to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
print('Libraries imported.')

Libraries imported.


#### Step 2-  Define Foursquare Credentials and Version.

#### The CLIENT_ID  and CLIENT_SECRET numbers has been taken off because of privacy 

In [27]:
CLIENT_ID = 'FTIL2DR5DOVWXSWQQBE3PNJNV4A3QVVT4LQCAE3I34NOZSNR' # your Foursquare ID
CLIENT_SECRET = 'OCB2DFICQSWK41QCHKV0JVJ05EXFTMW0HPL1EUN55A21WOFA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Step 3 -Working with only one boroughs that contain the word Toronto 

In [28]:
#getting the Borough that will be used
df_toro=toro_data[toro_data['Borough'].str.contains("Toronto")]
df_toro.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


##### creating a function that will localise available vanues in toronto 

In [29]:
#locate available vanues in toronto 
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Using the getNearbyVenues function created above to get the nearby Venues

In [30]:
toro_venues = getNearbyVenues(names=df_toro['Neighbourhood'],
                                   latitudes=df_toro['Latitude'],
                                   longitudes=df_toro['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [31]:
#examine the data
toro_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [32]:
#grouping by the Neighbourhood count
toro_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,59,59,59,59,59,59
Christie,15,15,15,15,15,15
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [33]:
#cheack the first 5 data and the shape
print(toro_venues.shape)
toro_venues.head()

(1600, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [34]:
#print out the uniques categories.
print('There are {} uniques categories.'.format(len(toro_venues['Venue Category'].unique())))

There are 236 uniques categories.


In [35]:
# Analyse each neighborhood
# one hot encoding
toro_onehot = pd.get_dummies(toro_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toro_onehot['Neighbourhood'] = toro_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toro_onehot.columns[-1]] + list(toro_onehot.columns[:-1])
toro_onehot = toro_onehot[fixed_columns]

toro_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#check the shape
toro_onehot.shape

(1600, 237)

In [37]:
#grouping the rows by neighbourhood and by taking the mean of the frequency of occurrence of each category
toro_grouped = toro_onehot.groupby('Neighbourhood').mean().reset_index()
toro_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.016949,0.000000,0.000000,0.016949,0.000000,0.016949
5,Christie,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012987,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025974
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
#check the shape after the grouping
toro_grouped.shape

(38, 237)

In [39]:
#print each neighbourhood along with the top 10 most common venues
num_top_venues = 10

for hood in toro_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toro_grouped[toro_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3         Cheese Shop  0.03
4      Farmers Market  0.03
5          Restaurant  0.03
6            Beer Bar  0.03
7            Pharmacy  0.03
8  Seafood Restaurant  0.03
9   French Restaurant  0.02


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.14
1          Breakfast Spot  0.09
2             Coffee Shop  0.09
3      Italian Restaurant  0.05
4                 Stadium  0.05
5  Furniture / Home Store  0.05
6               Nightclub  0.05
7            Climbing Gym  0.05
8                     Bar  0.05
9                  Bakery  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1            Comic Shop  0.06
2         Garden Center  0.06
3                  Park  0.06
4      Recording Stu

##### Creating a function to sort the venues in descending order

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Creating a new dataframe to display the top 10 venues for each neighborhood

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toro_grouped['Neighbourhood']

for ind in np.arange(toro_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toro_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Seafood Restaurant,Pharmacy,Cheese Shop,Restaurant,Juice Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Convenience Store,Furniture / Home Store,Burrito Place,Restaurant,Stadium,Italian Restaurant,Intersection
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Gym / Fitness Center,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Thai Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Juice Bar,Korean Restaurant
5,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Restaurant,Baby Store,Coffee Shop,Creperie
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Hotel,Fast Food Restaurant,Burger Joint,Pub
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Gym,American Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant
8,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Coffee Shop,Café,Brewery,Discount Store
9,Davisville North,Hotel,Gym / Fitness Center,Sandwich Place,Park,Breakfast Spot,Department Store,Food & Drink Shop,Donut Shop,Dog Run,Doner Restaurant


### Step 4  Clustering Neighborhoods

In [42]:
# set number of clusters
kclusters = 5

toro_grouped_clustering = toro_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toro_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toro_merged = df_toro

# merge toro_grouped with df_toro to add latitude/longitude for each neighborhood
toro_merged = toro_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toro_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,3.0,Neighborhood,Health Food Store,Trail,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Spa,Japanese Restaurant,Juice Bar
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0.0,Park,Sandwich Place,Fast Food Restaurant,Pet Store,Pub,Board Shop,Italian Restaurant,Fish & Chips Shop,Steakhouse,Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Park,Neighborhood
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4.0,Park,Bus Line,Swim School,Business Service,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [44]:
#checking the shape
toro_merged.shape

(39, 16)

##### Getting the coordinate of Toronto

In [45]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


#### creating a map of Toronto using the latitude and longitude values collected

In [46]:
to_map = folium.Map(location=[latitude, longitude], zoom_start=10)
to_map

In [47]:
#### The map with the Clusters 

In [48]:
to_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, borough, neighborhood in zip(
        toro_data['Latitude'], 
        toro_data['Longitude'], 
        toro_data['Borough'], 
        toro_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(to_map)  

to_map

### Step 5- Examining the 5 clusters

##### The first cluster

In [49]:
toro_merged.loc[toro_merged['Cluster Labels'] == 0, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Yoga Studio,Bubble Tea Shop,Spa,Japanese Restaurant,Juice Bar
42,East Toronto,0.0,Park,Sandwich Place,Fast Food Restaurant,Pet Store,Pub,Board Shop,Italian Restaurant,Fish & Chips Shop,Steakhouse,Restaurant
43,East Toronto,0.0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Park,Neighborhood
45,Central Toronto,0.0,Hotel,Gym / Fitness Center,Sandwich Place,Park,Breakfast Spot,Department Store,Food & Drink Shop,Donut Shop,Dog Run,Doner Restaurant
46,Central Toronto,0.0,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Chinese Restaurant,Café,Restaurant,Diner,Salon / Barbershop,Mexican Restaurant
47,Central Toronto,0.0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Gym,Italian Restaurant,Coffee Shop,Café,Brewery,Discount Store
49,Central Toronto,0.0,Coffee Shop,Bagel Shop,Fried Chicken Joint,Liquor Store,Sandwich Place,Restaurant,Supermarket,Bank,Sushi Restaurant,Pub
51,Downtown Toronto,0.0,Coffee Shop,Restaurant,Café,Bakery,Pet Store,Pub,Pizza Place,Pharmacy,Park,Italian Restaurant
52,Downtown Toronto,0.0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Hotel,Fast Food Restaurant,Burger Joint,Pub
53,Downtown Toronto,0.0,Coffee Shop,Bakery,Park,Breakfast Spot,Café,Pub,Theater,French Restaurant,Wine Shop,Health Food Store


##### The secord cluster

In [50]:

toro_merged.loc[toro_merged['Cluster Labels'] == 1, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,1.0,Home Service,Garden,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


##### The third cluster

In [51]:
toro_merged.loc[toro_merged['Cluster Labels'] == 2, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2.0,Park,Playground,Trail,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
64,Central Toronto,2.0,Park,Sushi Restaurant,Trail,Jewelry Store,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


##### The fourth cluster

In [52]:
toro_merged.loc[toro_merged['Cluster Labels'] == 3, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,3.0,Neighborhood,Health Food Store,Trail,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant


##### The fifth cluster

In [53]:
toro_merged.loc[toro_merged['Cluster Labels'] == 4, toro_merged.columns[[1] + list(range(5, toro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4.0,Park,Bus Line,Swim School,Business Service,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


The first cluster seems to have almost all the venues, this can been seen through the map

# The End of part 3, Thanks